In [3]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pprint import pprint

In [4]:
data = open('data.xml')
md = BeautifulSoup(data) # md - machines data

print(md.findAll("machine"))
# >>> y.foo.bar.type["foobar"]
# u'1'

# >>> y.foo.bar.findAll("type")
# [<type foobar="1"></type>, <type foobar="2"></type>]

# >>> y.foo.bar.findAll("type")[0]["foobar"]
# u'1'
# >>> y.foo.bar.findAll("type")[1]["foobar"]
# u'2'

[<machine yearofproduction="-2">
<summary delta="0" profit="0" servisecost="0" year="0"></summary>
<summary delta="0" profit="0" servisecost="0" year="1"></summary>
<summary delta="0" profit="0" servisecost="0" year="2"></summary>
<summary delta="250" profit="60" servisecost="55" year="3"></summary>
<summary delta="260" profit="60" servisecost="55" year="4"></summary>
<summary delta="270" profit="50" servisecost="55" year="5"></summary>
<summary delta="280" profit="50" servisecost="60" year="6"></summary>
<summary delta="280" profit="50" servisecost="60" year="7"></summary>
<summary delta="290" profit="40" servisecost="60" year="8"></summary>
<summary delta="290" profit="40" servisecost="60" year="9"></summary>
<summary delta="300" profit="40" servisecost="65" year="10"></summary>
<summary delta="300" profit="30" servisecost="65" year="11"></summary>
<summary delta="310" profit="30" servisecost="70" year="12"></summary>
</machine>, <machine yearofproduction="1">
<summary delta="200" pr

In [13]:
# get r(t) where y - year/number of period; t - age; y-t - year of production

def r(t, y):
    r = []
    try:
        if((t-y) >=0 and (t-y)<=2):
            r = '0'
        elif ((y-t) > 0):
            r = md.findAll("machine")[y-t].findAll("summary")[t]["profit"]
        elif ((y-t) < 0):
#             r = md.findAll("machine")[0].findAll("summary")[t-y]["profit"]
            r = '0'
    except IndexError:
        r = '0'
    
    return int(r)
    
r(6,3)

0

In [44]:
# get u(t) where y - year/number of period; t - age; y-t - year of production

def u(t, y):
    try:
        if((t-y) >=0 and (t-y)<=2):
            u = '0'
        elif ((y-t) == 3):
            u = md.findAll("machine")[0].findAll("summary")[t]["servisecost"]
        elif ((y-t) > 3):
            u = '0'
        else:
            u = md.findAll("machine")[y-t].findAll("summary")[t]["servisecost"]
#         elif ((y-t) < 0):
# #             u = '0'
#             u = md.findAll("machine")[0].findAll("summary")[t-y]["servisecost"]
    except IndexError:
        u = '0'
    
    return int(u)
    
# pprint(md.findAll("machine")[0].findAll("summary")[5]["servisecost"])
u_arr = [[u(t,y) for y in range(n)] for t in range(t_max)]
pprint(u_arr)
# u(5,2)

[[0, 20, 15, 0, 0, 0, 0, 0, 0],
 [0, 0, 20, 20, 0, 0, 0, 0, 0],
 [0, 0, 0, 25, 20, 0, 0, 0, 0],
 [0, 0, 0, 0, 25, 25, 55, 0, 0],
 [0, 0, 0, 0, 0, 30, 25, 55, 0],
 [0, 0, 0, 0, 0, 0, 30, 30, 55],
 [0, 0, 0, 0, 0, 0, 0, 35, 30],
 [0, 0, 0, 0, 0, 0, 0, 0, 40],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [65, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 70, 0, 0, 0, 0, 0, 0, 0]]


In [36]:
# get c(t) where y - year/number of period; t - age; y-t - year of production

def c(t, y):
    c = []
    try:
        if((t-y) >=0 and (t-y)<=2):
            c = '0'
        elif ((y-t) > 0):
            c = md.findAll("machine")[y-t].findAll("summary")[t]["delta"]
        elif ((y-t) < 0):
            c = '0'
#             c = md.findAll("machine")[0].findAll("summary")[t-y]["delta"]
    except IndexError:
        c = '0'
    
    return int(c)
    
c(4,5)

255

In [16]:
def findMax_t():
    arr = md.findAll("machine")

    max_t = len(md.findAll("machine")[0].findAll("summary"))
    for m in arr:
        m1 = m.findAll("summary")
        if max_t < len(m1):
            max_t = len(m1)
            
    return max_t
findMax_t()

13

In [17]:
n = 9
t_max = findMax_t()

w = [[{"w": 0, "x": False} for p in range(t_max+1)] for k in range(n+1)]
# pprint(w)

for i in reversed(range(n)):
    for t in range(t_max):
        save_profit = r(t,i) - u(t,i) + w[i + 1][t + 1]["w"]
        print('save_profit, r,u,w,i,t')
        print(save_profit,r(t,i), u(t,i), w[i + 1][t + 1]["w"],i,t)
        change_profit = r(0,i) - u(0,i) - c(t,i) + w[i + 1][t]["w"]
        print('change_profit, r,u,c,w,i,t')
        print(change_profit,r(0,i), u(0,i), c(t,i), w[i + 1][t]["w"],i,t)
        
        if save_profit >= change_profit:
            w[i][t]["w"] = save_profit
            w[i][t]["x"] = "sv"
        else:
            w[i][t]["w"] = change_profit
            w[i][t]["x"] = "ch"
            
for s in range(len(w)):
    w[s][s]["w"] = 0
    w[s][s]["x"] = False
    w[s][s+1]["w"] = 0
    w[s][s+1]["x"] = False
    w[s][s+2]["w"] = 0
    w[s][s+2]["x"] = False
    
w = w[::-1]
pprint(w)
       
    

save_profit, r,u,w,i,t
135 140 5 0 8 0
change_profit, r,u,c,w,i,t
-85 140 5 220 0 8 0
save_profit, r,u,w,i,t
115 125 10 0 8 1
change_profit, r,u,c,w,i,t
-85 140 5 220 0 8 1
save_profit, r,u,w,i,t
100 110 10 0 8 2
change_profit, r,u,c,w,i,t
-95 140 5 230 0 8 2
save_profit, r,u,w,i,t
95 110 15 0 8 3
change_profit, r,u,c,w,i,t
-90 140 5 225 0 8 3
save_profit, r,u,w,i,t
55 80 25 0 8 4
change_profit, r,u,c,w,i,t
-95 140 5 230 0 8 4
save_profit, r,u,w,i,t
55 80 25 0 8 5
change_profit, r,u,c,w,i,t
-125 140 5 260 0 8 5
save_profit, r,u,w,i,t
35 65 30 0 8 6
change_profit, r,u,c,w,i,t
-130 140 5 265 0 8 6
save_profit, r,u,w,i,t
20 60 40 0 8 7
change_profit, r,u,c,w,i,t
-135 140 5 270 0 8 7
save_profit, r,u,w,i,t
0 0 0 0 8 8
change_profit, r,u,c,w,i,t
135 140 5 0 0 8 8
save_profit, r,u,w,i,t
0 0 0 0 8 9
change_profit, r,u,c,w,i,t
135 140 5 0 0 8 9
save_profit, r,u,w,i,t
0 0 0 0 8 10
change_profit, r,u,c,w,i,t
135 140 5 0 0 8 10
save_profit, r,u,w,i,t
0 0 0 0 8 11
change_profit, r,u,c,w,i,t
135 14

In [18]:
from prettytable import PrettyTable
def print_table(input):
    #Color
    G = "\033[0;32;40m" # GREEN
    Y = "\033[0;33;40m" # Yellow
    N = "\033[0m" # Reset

    
    t = PrettyTable(['year(n) \ age (t)'] + [i for i in range(t_max+1)])

    row = []
    for k in range(len(input)):
        row.append('F' + str(len(input) - k-1) + '(t))')
        for w in input[k]:
            
            w_val = w['w']
            if w['x'] == 'sv':
                val = G + str(w_val) + N
            elif w['x'] == 'ch':
                val = Y + str(w_val) + N
            
            else:
                val = '---'
            
            row.append(val)
        if k!=0:    
            t.add_row(row)
        row = []

    
    print(t)
print_table(w)

+-------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| year(n) \ age (t) |  0  |  1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  |  9  |  10 |  11 |  12 |  13 |
+-------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|       F8(t))      | 135 | 115 | 100 |  95 |  55 |  55 |  35 |  20 | --- | --- | --- | 135 | 135 | --- |
|       F7(t))      | 240 | 210 | 195 | 125 | 120 |  70 |  55 | --- | --- | --- | 260 | 260 | 260 | --- |
|       F6(t))      | 325 | 300 | 205 | 195 | 115 |  95 | --- | --- | --- | 375 | 375 | 375 | 375 | --- |
|       F5(t))      | 410 | 300 | 275 | 165 | 135 | --- | --- | --- | 485 | 485 | 485 | 485 | 485 | --- |
|       F4(t))      | 400 | 365 | 225 | 185 | --- | --- | --- | 485 | 585 | 585 | 585 | 585 | 585 | --- |
|       F3(t))      | 460 | 295 | 240 | --- | --- | --- | 485 | 585 | 680 | 680 | 680 | 680 | 680 | --- |
|       F2(t))      | 380 | 305 | --- | --- | 